In [7]:
# Import libraries
import cv2
import pymongo
import gridfs
from datetime import datetime
import bson
import io
from datetime import datetime, timezone

In [8]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/") # default connection string
db = client["face_db"]
fs = gridfs.GridFS(db)

In [13]:
# Initialize webcam and face detector
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

print("[INFO] Press 's' to scan your face or 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Face Capture", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('s'):  # Save face on pressing 's'
        if len(faces) == 0:
            print("[WARN] No face detected.")
            continue

        # Assume only the first face
        (x, y, w, h) = faces[0]
        face_img = frame[y:y+h, x:x+w]

        # Encode image to memory
        is_success, buffer = cv2.imencode(".jpg", face_img)
        io_buf = io.BytesIO(buffer)

        # Store in MongoDB using GridFS
        metadata = {
            "uploaded_at": datetime.now(timezone.utc),
            "description": "User face image",
            "format": "jpg"
        }
        file_id = fs.put(io_buf.getvalue(), filename="face.jpg", metadata=metadata)
        print(f"[SUCCESS] Face stored in MongoDB with file ID: {file_id}")

    elif key == ord('q'):
        break

# Release everything
cap.release()
cv2.destroyAllWindows()

[INFO] Press 's' to scan your face or 'q' to quit
[SUCCESS] Face stored in MongoDB with file ID: 687028b9b54b00f962b6db3a
